# Разработка дашборда в Yandex DataLens для визуализации отчета главного онколога РФ

## Описание проекта, цель и задачи
Основная цель проекта заключается в создании дашборда для визуализаци медицинских данных, собранных в рамках ежегодного отчета онколога РФ за 2021 год. В дальнейшем планируется работа с данными за 2022 и предыдущие годы. Проект носит гуманитарную составляющую, так как призван облегчить анализ данных в медицинской сфере для понимания статистики по раковым заболеваниям в Российской Федерации.

### Задачи:
- Импортировать, предобработать и выгрузить данные из отчета
- Подключить полученные предобработанные данные к Yandex DataLens, сформировать чарты
- На основе чартов спроектировать дашборд с соответствующими селекторами
- Подготовить краткую документацию к этапам предобработки данных, а также логике построения дашборда

### Особенности данных:
Все данные были получены из отчетов в формате PDF, поэтому представляют собой удобную форму для чтения пользователем, но не совсем удобны для анализа. Решения, принятые в рамках предобработки, представлены в следующем блоке.

## Особенности предобработки данных

### В ходе предобработки данных были сделаны следующие допущения:
1. Так как количество листов в файлах разное, то было принято решение обработать первые листы отдельно от вторых. На базе полученных преобразований получить 2 вида общих таблиц при помощи метода concat(): первого листа и второго.
2. В наборе таблиц присутствует одна, которая выбивается среди всех (22). Оставил ее вне рамок анализа и дашборда, так как не совсем понял, куда ее выносить. Тем не менее, думаю, что код ниже сможет предобработать и ее, только не в цикле, а вне него.
3. Сделал колонку с кодами, но в некоторых случаях соответствия между кодом и названием органа не было. В целом, нужно получить список соответствий от заказчика и совместить по ключу (в нашем случае - коду), с этим проблем не должно быть. Оставляю пока что так из-за нехватки времени.
4. Весь проект делал локально, отсюда и выбор библиотек (тетрадка лежит в одной директории со всеми файлами).

## Проектирование дашборда

Ссылка на дашборд - https://datalens.yandex/6jg11tbu24ayv

## Предобработка данных

In [1]:
# импортируем все необходимые библиотеки
import pandas as pd
import openpyxl
import numpy as np
from pathlib import Path
import re

In [2]:
# указываем локальный путь до директории, где лежит Jupyter-notebook, там же лежат и файлы
input_dir = Path.cwd()

# создаем два пустых массива, куда будем складывать первые и вторые листы всех таблиц
df_sheet_one = []
df_sheet_two = []

In [3]:
# объявим функцию для отделения федеральных округов от областей
def set_district(row):
    if row['область'].isupper():
        return row['область']
    else:
        return np.nan

In [4]:
# объявим функцию для предобработки данных
def create_df_from_xlsx(sheetname):
    # принты использую для отладки кода, чтобы понимать, в каком файле произошла ошибка, сохраним датафрейм
    print(path)
    df = pd.read_excel(path, sheetname)
    # избавимся от шапки таблицы (для этого напишем универсальное решение, ориентироваться будем на строчку, перед словом
    # РОССИЯ, так как все файлы единообразны), перед этим вынесем код в отдельную колонку
    df['код'] = re.search(r'\(([А-Яа-яA-Za-z0-9-,;]+)\)', df.columns[0])[1]
    condition = df[df.iloc[:, 0] == 'РОССИЯ'].index[0] - 1
    df = df.drop(df.index[0: condition])
    df.columns = df.loc[condition]
    df = df.drop(condition).reset_index(drop=True) 
    return df

In [5]:
# создадим списки для переименования обоих листов в дальнейшем
first_sheet_columns = ['область', 'всего на учете', 'в т.ч. выявлены активно, %', 'учет: абсолют.', 'учет: на 100 тыс.', 'учет 5 лет: абсолют.',\
                          'учет 5 лет: % от учета', 'индекс накопления контингентов', 'летальность, %', 'код']
second_sheet_columns = ['область', 'всего ЗНО', 'диагноз подтвержден морфологически', 'стадия I', 'стадия II', 'стадия III',\
                          'стадия IV', 'неизвестная стадия', 'летальность в 1 год', 'код']

In [6]:
# пройдемся циклом по всем файлам формата xsl в указанной ранее директории без доступа в директории уровнем ниже, в зависимости от
# количества листов в файле проведем необходимую предобработку (прокомментирую логику только первой ветки, остальные две по аналогии)
for path in list(input_dir.glob('*.xls*')):
    # открываем файл и получаем массив с названиями листов, чтобы автоматизировать выбор названия листа
    workbook = openpyxl.load_workbook(path)
    sheetnames = workbook.sheetnames
    if len(sheetnames) > 1:
        if sheetnames[0]:
            df = create_df_from_xlsx(sheetnames[0])
            
            # преобразуем заголовки в понятную форму
            df.columns = first_sheet_columns

            # на основе созданной функции создадим столбец с федеральными округами, а также удалим строчки с суммарными значениями
            # по каждому федеральному округу из датафрейма
            df['ФО'] = df.apply(set_district, axis=1).fillna(method='ffill')
            df = df.drop(df.loc[df['область'].str.isupper()].index)

            # полученный датафрейм прикрепляем к массиву
            df_sheet_one.append(df)

        if sheetnames[1]:
            print(path)
            df = create_df_from_xlsx(sheetnames[1])
            df.columns = second_sheet_columns
            df['область'] = df['область'].astype('str')
            df['ФО'] = df.apply(set_district, axis=1).fillna(method='ffill')
            df = df.drop(df.loc[df['область'].str.isupper()].index)
            df_sheet_two.append(df)

    else:
        print(path)
        df = create_df_from_xlsx(sheetnames[0])
        df.columns = first_sheet_columns
        df['область'] = df['область'].astype('str')
        df['ФО'] = df.apply(set_district, axis=1).fillna(method='ffill')
        df = df.drop(df.loc[df['область'].str.isupper()].index)
        df_sheet_one.append(df)

/Users/paulsapozhnikov/Desktop/Analytics/test/2021_Таблица_045_Состояние_онко_помощи_в_РФ.xlsx
/Users/paulsapozhnikov/Desktop/Analytics/test/2021_Таблица_045_Состояние_онко_помощи_в_РФ.xlsx
/Users/paulsapozhnikov/Desktop/Analytics/test/2021_Таблица_045_Состояние_онко_помощи_в_РФ.xlsx
/Users/paulsapozhnikov/Desktop/Analytics/test/2021_Таблица_033_Состояние_онко_помощи_в_РФ.xlsx
/Users/paulsapozhnikov/Desktop/Analytics/test/2021_Таблица_033_Состояние_онко_помощи_в_РФ.xlsx
/Users/paulsapozhnikov/Desktop/Analytics/test/2021_Таблица_033_Состояние_онко_помощи_в_РФ.xlsx
/Users/paulsapozhnikov/Desktop/Analytics/test/2021_Таблица_040_Состояние_онко_помощи_в_РФ.xlsx
/Users/paulsapozhnikov/Desktop/Analytics/test/2021_Таблица_040_Состояние_онко_помощи_в_РФ.xlsx
/Users/paulsapozhnikov/Desktop/Analytics/test/2021_Таблица_040_Состояние_онко_помощи_в_РФ.xlsx
/Users/paulsapozhnikov/Desktop/Analytics/test/2021_Таблица_036_Состояние_онко_помощи_в_РФ.xlsx
/Users/paulsapozhnikov/Desktop/Analytics/test/2021

In [7]:
# объединим датафреймы каждого из листов по вертикали, по аналогии с UNION-объединением в SQL
sheet_one = pd.concat(df_sheet_one, ignore_index=False).reset_index(drop=True).reset_index()
sheet_two = pd.concat(df_sheet_two, ignore_index=False).reset_index(drop=True).reset_index()

In [8]:
# экспортируем оба файла в формат .csv, так как Yandex DataLens не поддерживает формат Excel
sheet_one.to_csv('sheet_one.csv', index=True)
sheet_two.to_csv('sheet_two.csv', index=True)
sheet_two.to_excel('sheet_two.xlsx', index=True)